In [ ]:
import xarray as xr
import pandas as pd
import numpy as np

# Load your bounding box data from a CSV file
# This CSV file contains geographic coordinates of bounding boxes (MinLon, MinLat, MaxLon, MaxLat)
bbox_file_path = r"bounding_boxes_tower.csv"
bbox_data = pd.read_csv(bbox_file_path)

# Load the xarray dataset from which we want to extract the variable data
# This dataset should contain variables like 'WATER', 'BARE', 'BUILT', etc. at various spatial points (lon, lat)
# Replace 'your_dataset_file.nc' with the actual file path of your dataset
ds = xr.open_dataset(r"your_dataset_file.nc")

# List of variables to extract from the xarray dataset
variables = ['WATER', 'BARE', 'BUILT', 'GRASS-MAN', 'GRASS-NAT', 'SHRUBS-BD', 'TREES-BD', 
             'SHRUBS-BE', 'TREES-BE', 'LAND', 'SHRUBS-ND', 'TREES-ND', 'SHRUBS-NE', 'TREES-NE', 'SNOWICE']

# Initialize new columns in the bbox_data DataFrame for each variable, setting initial values to NaN
for variable in variables:
    bbox_data[variable] = np.nan  # Create a new column for each variable with default value NaN

# Function to calculate the center point of a bounding box
# Takes minimum and maximum longitude and latitude and returns the center point
def get_center_point(min_lon, min_lat, max_lon, max_lat):
    center_lon = (min_lon + max_lon) / 2  # Calculate center longitude
    center_lat = (min_lat + max_lat) / 2  # Calculate center latitude
    return center_lon, center_lat

# Iterate over each row (bounding box) in the bbox_data DataFrame
for index, row in bbox_data.iterrows():
    # Calculate the center point (latitude, longitude) of the bounding box
    center_lon, center_lat = get_center_point(row['MinLon'], row['MinLat'], row['MaxLon'], row['MaxLat'])
    
    # Select the nearest data point from the xarray dataset for the given center point coordinates
    # Using the 'nearest' method to find the closest point in the dataset based on lat/lon
    data_point = ds.sel(lon=center_lon, lat=center_lat, method='nearest')
    
    # Update the DataFrame with values for each variable at the nearest point
    for variable in variables:
        # Extract the value for the variable from the xarray dataset at the nearest point
        # .item() ensures the result is a scalar (single value), and we check if the variable has data at that point
        bbox_data.at[index, variable] = data_point[variable].values.item() if data_point[variable].values.size else np.nan

# Save the updated DataFrame (with extracted variable data) to a new CSV file
output_file_path = r'A:\Research\2_NEON\11_PFT\pft_tower.csv'
bbox_data.to_csv(output_file_path, index=False)  # Save the DataFrame without row indices

# Print a message indicating that the data has been successfully saved
print(f"Updated data saved to {output_file_path}")
